In [6]:
# ============================================================
#  DATA SUMMARY + EDA SONG NGỮ (TẠO FILE WORD CÓ BIỂU ĐỒ THẬT)
# ============================================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches
from io import BytesIO
import os

# === 1. Đường dẫn file ===
file_path = r"C:\Users\BOOK 1\Downloads\Data Stata3.xlsx"
output_path = os.path.join(os.path.dirname(file_path), "Data_Summary_EDA.docx")

# === 2. Đọc & gom tất cả sheet về dạng long ===
xls = pd.ExcelFile(file_path)
all_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df = df.dropna(how="all")
    df = df.rename(columns={df.columns[0]: "Province"})
    df_long = df.melt(id_vars="Province", var_name="Year", value_name="Value")
    df_long["Indicator"] = sheet.strip()
    all_list.append(df_long)

panel_long = pd.concat(all_list, ignore_index=True)

# === 3. Làm sạch dữ liệu ===
panel_long["Province"] = panel_long["Province"].astype(str).str.strip()
panel_long["Year"] = panel_long["Year"].astype(int)
panel_long["Value"] = (
    panel_long["Value"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.extract(r"([-+]?\d*\.?\d+)")
)
panel_long["Value"] = pd.to_numeric(panel_long["Value"], errors="coerce")

# Pivot wide
panel_wide = panel_long.pivot_table(
    index=["Province", "Year"], columns="Indicator", values="Value", aggfunc="mean"
).reset_index()
panel_wide.columns.name = None

# === 4. Thống kê mô tả ===
summary = panel_wide.describe().transpose()

# === 5. Tạo file Word ===
doc = Document()
doc.add_heading("Data Summary and Exploratory Data Analysis (EDA)", 0)

# ----- Mô tả tổng quan -----
doc.add_heading("1. Data Summary / Tóm tắt dữ liệu", level=1)
doc.add_paragraph(
    "This dataset contains socio-economic indicators of 63 provinces in Vietnam "
    "from 2019 to 2023, collected from the General Statistics Office (GSO)."
)
doc.add_paragraph(
    "Bộ dữ liệu bao gồm các chỉ tiêu kinh tế - xã hội của 63 tỉnh/thành Việt Nam "
    "giai đoạn 2019–2023, được thu thập từ Tổng cục Thống kê (GSO)."
)

# ----- Bảng thống kê -----
doc.add_heading("Summary Statistics / Thống kê mô tả", level=2)
table = doc.add_table(rows=1, cols=summary.shape[1] + 1)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = "Variable"
for i, col in enumerate(summary.columns):
    hdr_cells[i + 1].text = col
for var, row in summary.iterrows():
    row_cells = table.add_row().cells
    row_cells[0].text = str(var)
    for i, val in enumerate(row):
        row_cells[i + 1].text = f"{val:.2f}" if pd.notna(val) else ""

# === 6. Biểu đồ ===
plots = {}

# GDP bình quân đầu người theo năm
if "gdp_nguoi" in panel_wide.columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=panel_wide, x="Year", y="gdp_nguoi")
    plt.title("GDP per Capita by Year / GDP bình quân đầu người theo năm")
    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    plots["gdp_year"] = buf
    plt.close()

# Tỷ lệ nghèo vs GDP
if "ty_le_ho_ngheo" in panel_wide.columns and "gdp_nguoi" in panel_wide.columns:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(data=panel_wide, x="gdp_nguoi", y="ty_le_ho_ngheo", hue="Year", palette="viridis")
    plt.title("Poverty vs GDP / Tỷ lệ hộ nghèo và GDP bình quân đầu người")
    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    plots["poverty_gdp"] = buf
    plt.close()

# Ma trận tương quan
corr = panel_wide.select_dtypes("number").corr()
plt.figure(figsize=(7, 5))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap / Ma trận tương quan")
plt.tight_layout()
buf = BytesIO()
plt.savefig(buf, format="png")
buf.seek(0)
plots["heatmap"] = buf
plt.close()

# === 7. Chèn biểu đồ vào Word ===
doc.add_heading("2. Exploratory Data Analysis / Phân tích khám phá dữ liệu", level=1)
doc.add_paragraph(
    "The following figures show the distribution and relationships of selected indicators."
)
doc.add_paragraph(
    "Các biểu đồ dưới đây thể hiện phân bố và mối quan hệ giữa một số chỉ tiêu tiêu biểu."
)

for title, buf in plots.items():
    doc.add_picture(buf, width=Inches(5.8))
    doc.add_paragraph(title.replace("_", " ").capitalize())

# === 8. Lưu file ===
doc.save(output_path)
print(f"✅ File Word đã được tạo thành công: {output_path}")

✅ File Word đã được tạo thành công: C:\Users\BOOK 1\Downloads\Data_Summary_EDA.docx


In [5]:
pip install python-docx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
